#### Lab2 - Data Collection and Pre-processing - Manu Mathew - 8990691

##### Adding the required imports 

In [7]:
import pandas as pd
from dataclasses import dataclass

##### Load raw CSV, display first 3 rows

In [8]:
df = pd.read_csv("data/Grocery_Transactions.csv")
print(df.head(3))

         date  customer_id   product  product_id  price  quantity  \
0  2025-05-26         3000    Apples           1  13.23         3   
1  2025-05-04         3001      Milk           4  27.94         1   
2  2025-05-19         3002  Tomatoes           9  13.72         6   

  shipping_city  coupon_id  
0        Ottawa          1  
1     Vancouver          4  
2        Ottawa          5  


##### Justify dict vs namedtuple vs class
- Dictionary: Mutable, unordered collection of key-value pairs.
- NamedTuple: Similar to tuple, but it is object with named fields.
- Class: A blueprint for creating objects with attributes and behaviors.

I have chosen class, so that I can include the implementations of ingest, wrangle, clean, transform, feature-engineer and serialization methods, so that the codebase is more scalable.

##### Implement Transaction class and use it to populate data structure an object-oriented data structure

In [9]:
@dataclass
class Transaction:
    date: str
    customer_id: str
    product:str
    product_id: str
    price: float
    quantity: int
    shipping_city: str
    coupon_id:int
